In [1]:
print('Hello World, begin project')

Hello World, begin project


# Problem statemant:

1. Extract the news articles links of last 30 days using python

2. From each link extract the Title, Body, date.

3. Clean the text body and remove the noise in it

4. Create the DB named news and table named articles in the DB (any DB is okay)

4. Save the fields in the Database table and Excel sheet
***
Data Source link: https://www.moneycontrol.com/
***

### Importing libraries

In [2]:
import os
from time import sleep

# Libraries for web Scraping
import requests
from bs4 import BeautifulSoup  

# Libraries for date operations
from dateutil import parser  
from datetime import timedelta
import datetime

# Libraries for text cleaning and niose removal
import unidecode
import re

# Libraries for export to database and excel
from pymongo import MongoClient
from openpyxl import load_workbook
import xlsxwriter

### User-Defined Functions

In [25]:
def get_art_list_from_page (url: str) :
    '''
    Function to extract the links of all news article from a page (each page contains about 25 articles).
    
    | Input: URL to all news page with page number
    | Output: 1) List of article URLs in given page
            2) Publication date of the last article in the given page (to check for end of scraping) 
    '''

    art_url_list =[]
    response = requests.get(url)

    article_list = BeautifulSoup(response.text,'html.parser') # parse the news page
    links = article_list.find_all('li',attrs={'class':'clearfix'}) # extract all tags with article links
    
    for link in links:
        art_url_list.append(link.find('a').get("href")) # get link of article 
        last_article_date =  parser.parse(link.find('span').get_text(),ignoretz=True).date() # get date of published article
    
    return art_url_list, last_article_date

In [4]:
def text_cleaner(input_text:str):
    ''' 
    Function for noise removal and text cleaning.
    Operations: lower casing, remove non-unicode characters, remove links, remove numbers, remove extra spaces. 

    | Input: raw text
    | Output: processed text
    '''
    
    input_text = unidecode.unidecode(input_text) # removes non-breaking white spaces

    input_lower = input_text.lower() # lower case all characters

    clean1 = re.sub(r"(@[a-z0-9]+)|([^a-z \t])|(\w+:\/\/\S+)|\bwww\.[^\s]+|http.+?", "", input_lower)
    # @[A-Za-z0-9] matches @ mentions, email addresses
    # [^0-9A-Za-z \t] matches non alphabet characters
    # \w+:\/\/\S+ matches URLs
    # \bwww\.[^\s] matches web addresses starting with www
    # http.+? matches URL starting with http

    processed_text = re.sub("(\s){2,}", " ", clean1) # remove extra spaces

    return processed_text

In [5]:
def get_article_page (art_url:str):
    ''' 
    Function to extract and process news article from URL of article. 
    Extracted parameters: Title of article, Publication date, Text body of article. 
    Text body is further cleaned using text_cleaner() function.
    Returns dictionary which is ready for export and data storage. 
    Returns empty dictionary if the URL is not a news article or if any error is encountered during parsing.

    | Input: URL of article
    | Output: Dictionary with following keys: 'URL', 'Title', 'Publish_Date', Content' 
            Empty dictionary returned if any anomolous cases(mentioned above) are found
    '''

    try:
                
        article_full_content = []
        get_article = requests.get(art_url)
        
        article_soup_obj = BeautifulSoup(get_article.content,'html.parser') # parse page 
        article_title = article_soup_obj.find("h1",attrs={"class":"article_title artTitle"}).get_text() # get article title
        article_date =  article_soup_obj.find("div",attrs={"class":"article_schedule"}).find('span').get_text() # get article date
        article_body =  article_soup_obj.find("div",attrs={"class":"content_wrapper arti-flow"}) # locate article content
        article_paragraphs = article_body.findAll("p") # locate paragraphs in article content
        for i in article_paragraphs:
            article_full_content.append(i.get_text()) # extract text from paragraph

        article_full_content = ' '.join(article_full_content) # combine all paragraph contents
        
        # call text_cleaner() for processing the text content
        article_full_content = text_cleaner(article_full_content) 

        # create dictionary for export
        export_dict = {'URL': art_url, 'Title': article_title, 'Publish_Date': article_date, 'Content': article_full_content}
        return export_dict    
    except:
        return {}

In [6]:
def export_to_db(dict_to_db:dict, DB_name:str, table_name:str):
    ''' 
    Function to export dictionary containing article details to MongoDB. 

    | Input: dictionary for export, Database name, Table name
    '''

    client=MongoClient('mongodb://192.168.1.2:27017') # connect to database
    db=client[DB_name] # access database, if not exist then create database
    collection = db[table_name] # access table/collection, create it not exist

    collection.insert_one(dict_to_db) # store dictionary in table/collection

In [7]:
def export_to_excel(dict_to_xlsx:dict, WB_name:str, sheet_name:str):
    ''' 
    Function to export dictionary containing article details to Excel workbook.
    Each function call will append data to workbook.

    | Input: dictionary for export, Workbook name, Sheet name
    '''

    if WB_name[-5:] != '.xlsx': # add extension to workbook name
        WB_name = WB_name + '.xlsx'

    headers = ['URL','Title','Publish_Date','Content'] # Column headers
    
    if not os.path.isfile(WB_name): # create workbook if it does not exist
        book = xlsxwriter.Workbook(WB_name)
        sheet = book.add_worksheet(sheet_name)
        for (idx, header) in enumerate(headers):
            sheet.write(0, idx, header)
        book.close()
    
    loaded_book = load_workbook(WB_name)
    loaded_sheet = loaded_book[sheet_name]

    values = [dict_to_xlsx[key] for key in headers] # extract values for each key from dictionary 
    
    loaded_sheet.append(values) # write to excel file
    loaded_book.save(filename=WB_name)

### Main function

In [26]:
run_start_date = datetime.datetime.now().date() # date at time of program execution
page_last_article_date = datetime.datetime.now().date() # variable definition to enter loop, will be updated after entering loop 
last_date_for_extraction = run_start_date - timedelta(days=0) # last date till when article must be extracted 

# variable initializations
page_no = 1
full_list_art = [] 
total_num_of_articles = 0

while page_last_article_date >= last_date_for_extraction: # main loop | checking date for end condition
    url = 'https://www.moneycontrol.com/news/news-all/page-' + str(page_no) + '/'  # All News page URL of news website with page number  
    
    art_url_list, page_last_article_date = get_art_list_from_page(url) # get list of URLs from given page, get publish date of last article in given page

    for art_url in art_url_list: # loop to iterare elements in list of article URL 

        export_dict = get_article_page(art_url) # Scraping article URL for data

        if bool(export_dict): # check if empty dictionary
            total_num_of_articles += 1
            export_to_db(export_dict, DB_name='News', table_name='Articles') # export to MongoDB
            export_to_excel(export_dict, WB_name='News', sheet_name='Articles') # export to Excel file 
            
    print(page_last_article_date, '|| Page no: ', page_no, end="\r")
    page_no += 1  # move to next page in article list page 

print('Total number of articles processed: ', total_num_of_articles)